# Structured Outputs with Foundation AI Models

Thanks to the power of constrained decoding, Foundation AI's 16k instruct model can now produce structured outputs that adhere to complex [JSON schemas](https://json-schema.org/).

Constrained decoding is a technique for guiding the output of large language models. It modifies the probably distribution produced by the model, thereby only allowing tokens that are valid for the desired schema. There are many popular, open-source libraries for constrained decoding, such as [Guidance](https://github.com/guidance-ai/guidance) and [xGrammar](https://github.com/mlc-ai/xgrammar).

## How to generate structured outputs

Generating structured outputs is simple.

Start by creating a [JSON schema](https://json-schema.org/) for your desired output. Then, include the schema in your request, as shown below:
```
{
    "messages": [...],
    "response_format": {
        "type": "json_schema",
        "json_schema": {
            "name": "response_schema",
            "schema": <your_schema_here>,
        },
    }
}
```

In [1]:
import os
import requests